#European RosettaCon 2022 - Docking Session

###Theoretical aspects

As more and more protein structures are determined experimentally using X-ray crystallography or nuclear magnetic resonance (NMR) spectroscopy, molecular docking is increasingly used as a tool in **drug discovery**.

**Molecular docking simulations** explore the potential binding poses of small molecules on the **binding site** of a target protein for which an experimentally determined structure is available. 
Docking against protein targets generated by **comparative modelling** also becomes possible for proteins whose structures are yet to be solved.

Thus, the **_druggability_** of different compounds and their binding affinity on a given protein target can be calculated for further lead optimization processes.

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/docking_01.png'/>
<figcaption>FIGURE 1. In molecular docking, binding is evaluated in two steps: A) Energetics of the transition of the unbound states of ligand and target towards the conformations of the bound complex; and B) energetics of protein-ligand binding in these conformations. <br> Huey R et al (2007) <i>J Comput Chem 28(6), 1145-1152.</i></figcaption></center>
</figure>

Molecular docking programs perform a **search algorithm** in which varying conformations of a given ligand, typically generated using Monte Carlo or Genetic algorithms, are recursively evaluated until convergence to an energy minimum is reached. Finally, through an **affinity scoring function**, a ΔG [binding free energy in kcal/mol] is estimated and employed to rank the candidate poses as the sum of several energetic contributions (electrostatics, van der Waals, desolvation, etc).

#Part I – Downloading inputs

In [ ]:
#Here we download the input files from Github
!wget https://speicherwolke.uni-leipzig.de/index.php/s/Mw2EjEYHmqkp4zo/download/ligand_docking.zip
#This remove command is just in case you had some previous input files
!rm -rf input_files
#Here, we unzip the downloaded files
!unzip ligand_docking.zip

#Part II – Setting/Starting PyRosetta

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Notebook setup
import sys
if 'google.colab' in sys.modules:
  !pip install pyrosettacolabsetup
  import pyrosettacolabsetup
  pyrosettacolabsetup.install_pyrosetta()
  exit()

In [ ]:
# Notebook setup
import sys
if 'google.colab' in sys.modules:
    !pip install pyrosettacolabsetup
    import pyrosettacolabsetup
    pyrosettacolabsetup.mount_pyrosetta_install()
    print ("Notebook is set for PyRosetta use in Colab.  Have fun!")

from pyrosetta import *
from rosetta.protocols.rosetta_scripts import *
from pyrosetta import (
    init, pose_from_sequence, pose_from_file, Pose, MoveMap, create_score_function, get_fa_scorefxn,
    MonteCarlo, TrialMover, SwitchResidueTypeSetMover, PyJobDistributor,
)

# Part III – Preparing and running the docking  protocols


1. We start by setting up PyRosetta in Google Colab, based on your previously compiled and installed PyRosetta readily available in your Google Drive

**⚠️WARNING:** Remember that you MUST have compiled and installed PyRosetta before this tutorial!

2. In the following cell code, we import the PyRosetta modules that we are going to use

#### Docking ETQ

1. In the following cell code we give PyRosetta all the needed **Flags**. These flags are equivalent to command-line instructions in order to set the path for files and set the value of different (and numerous) options. We pass them into PyRosetta, thus initializing it


```
pyrosetta.distributed.init(flags)
```



In [3]:
#Flags are equivalent to command-line instructions in order to set the path for files and set the value of different options 
ligand_params = "/content/ligand_docking/1_vanilla_docking/answers/ligand_prep/ETQ.params"
flags = f"""
-extra_res_fa {ligand_params}
-packing:ex1
-packing:ex2
-packing:no_optH false
-packing:flip_HNQ true
-ignore_unrecognized_res 1
-packing:ignore_ligand_chi true
-out:path:all /content/out
-overwrite
-corrections::restore_talaris_behavior
"""
pyrosetta.distributed.init(flags)


2. One of the crucial concepts in PyRosetta are **poses**. They contain the information, sequence or structure, to which the protocol will be applied.



In [ ]:
pose = pyrosetta.io.pose_from_file(filename="/content/ligand_docking/1_vanilla_docking/answers/docking/3PBL_A_ETQ.pdb")
scorefxn = pyrosetta.create_score_function("ligand.wts")

3. While there are multiple ways of creating a protocol in Rosetta, in our case we are going to create a protocol from a **XML object**.



In [ ]:
#Creating a Rosetta protocol from a XML object
#Note that in the <MOVERS> we specify the fragment files and the threaded_pdb!!

%%time
xml = pyrosetta.rosetta.protocols.rosetta_scripts.XmlObjects.create_from_string("""
<ROSETTASCRIPTS>
		<SCOREFXNS>
			<ScoreFunction name="ligand_soft_rep" weights="ligand_soft_rep">
			</ScoreFunction>
			<ScoreFunction name="hard_rep" weights="ligand">
			</ScoreFunction>
		</SCOREFXNS>
    <RESIDUE_SELECTORS>
      <Chain name="chX" chains="X"/>
    </RESIDUE_SELECTORS>
    <SIMPLE_METRICS>
      <RMSDMetric name="rmsd_chX" residue_selector="chX" reference_name="store_native" residue_selector_ref="chX" robust="true" rmsd_type="rmsd_all" />
    </SIMPLE_METRICS>
		<LIGAND_AREAS>
			<LigandArea name="inhibitor_dock_sc" chain="X" cutoff="6.0" add_nbr_radius="true" all_atom_mode="false"/>
			<LigandArea name="inhibitor_final_sc" chain="X" cutoff="6.0" add_nbr_radius="true" all_atom_mode="false"/>
			<LigandArea name="inhibitor_final_bb" chain="X" cutoff="7.0" add_nbr_radius="false" all_atom_mode="true" Calpha_restraints="0.3"/>
		</LIGAND_AREAS>
		<INTERFACE_BUILDERS>
			<InterfaceBuilder name="side_chain_for_docking" ligand_areas="inhibitor_dock_sc"/>
			<InterfaceBuilder name="side_chain_for_final" ligand_areas="inhibitor_final_sc"/>
			<InterfaceBuilder name="backbone" ligand_areas="inhibitor_final_bb" extension_window="3"/>
		</INTERFACE_BUILDERS>
		<MOVEMAP_BUILDERS>
			<MoveMapBuilder name="docking" sc_interface="side_chain_for_docking" minimize_water="false"/>
			<MoveMapBuilder name="final" sc_interface="side_chain_for_final" bb_interface="backbone" minimize_water="false"/>
		</MOVEMAP_BUILDERS>
		<SCORINGGRIDS ligand_chain="X" width="15">
			<ClassicGrid grid_name="classic" weight="1.0"/>
		</SCORINGGRIDS>
		<MOVERS>
      <SavePoseMover name="spm" restore_pose="0" reference_name="store_native"/>
			<Transform name="transform" chain="X" box_size="7.0" move_distance="0.2" angle="20" cycles="500" repeats="1" temperature="5"/>
			<HighResDocker name="high_res_docker" cycles="6" repack_every_Nth="3" scorefxn="ligand_soft_rep" movemap_builder="docking"/>
			<FinalMinimizer name="final" scorefxn="hard_rep" movemap_builder="final"/>
			<InterfaceScoreCalculator name="add_scores" chains="X" scorefxn="hard_rep" native="/content/ligand_docking/1_vanilla_docking/docking/crystal_complex.pdb"/> 
		</MOVERS>
    <FILTERS>      
      <SimpleMetricFilter name="rmsd_chX" metric="rmsd_chX" cutoff="999999." comparison_type="lt" confidence="0"/>
    </FILTERS>
		<PROTOCOLS>
      <Add mover="spm"/>
			<Add mover_name="transform"/>
			<Add mover_name="high_res_docker"/>
			<Add mover_name="final"/>
			<Add mover_name="add_scores"/>
      <Add filter="rmsd_chX"/>
		</PROTOCOLS>
</ROSETTASCRIPTS>
    """).get_mover("ParsedProtocol") 

4. Once our protocol is set up, in the following code cells:
  * We create the outputs directory
  * We call the PyJobDistrubutor and pass three arguments
    * A name prefix to output our models as PDB files
    * The number of models to generate
    * The score function to employ(ligands)
  * We pass our pose to the job distributor that we just created
  * Then, we create a Pandas dataframe to store the score results
  * Finally, we loop while the jobs are not completed

In [ ]:
import pandas as pd

if not os.getenv("DEBUG"):
    working_dir = os.getcwd()
    output_dir = "outputs"
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    os.chdir(output_dir)

    jd = pyrosetta.toolbox.py_jobdistributor.PyJobDistributor(pdb_name="3PBL_ETQ",
                                                              nstruct=5,
                                                              scorefxn=scorefxn)
    jd.native_pose = pose
    df = pd.DataFrame()
    while not jd.job_complete:
        test_pose = pose.clone()
        xml.apply(test_pose)
        test_df = pd.DataFrame.from_records(dict(test_pose.scores), index=[jd.current_name])
        df = df.append(test_df)
        jd.output_decoy(test_pose)
    os.chdir(working_dir)

#Part IV – Analyze and visualize the modelled structures

1. Now, we need to determine our best models by analyzing their score dataframes. For this, we are going to obtain the data frame for each of the models that we generated with PyRosetta.

  We can visually inspect these data frames just writing their name in a code cell.

In [ ]:
df

In [ ]:
#Call here the name of the different dataframes that were created
# iterating the columns
for col in df.columns:
    print(col)

2. As Rosetta uses an stochastic algorithm for comparative modelling, we are only interested in the low energy structures. Thus, we can sort our models by the column total_score as in the following cell code:

In [ ]:
#hdf.sort_values('total_score')
df.sort_values('rmsd_chX')

3. Having too many tables or tables with too much information would be challenging to analyze our results. For simplicity, we can make a plot with the **total_score** of each PDB for each species using **matplotlib**

In [ ]:
import matplotlib
import seaborn
frames = [df]
results = pd.concat(frames)
matplotlib.rcParams['figure.figsize'] = [12.0, 8.0]
seaborn.barplot(x="rmsd_chX", y="index", data=results.sort_values('rmsd_chX').reset_index())

4. Finally, we can take a look at our best models using **py3Dmol**

In [ ]:
!pip install py3Dmol
import py3Dmol
view=py3Dmol.view()
#Loading template
view.addModel(open('/content/outputs/3PBL_ETQ_YOURBESTMODEL".pdb', 'r').read(),'pdb')
view.zoomTo({'chain':'X'})
view.setBackgroundColor('white')
view.setStyle({'chain':'A'},{'cartoon': {'color':'purple'}})
#See residues that are a distance X from the residue 158
view.addStyle({'within':{'distance': 5,
                         'sel':{'chain':'X'}
                         }
               }
              ,{'stick':{'colorscheme':'greenCarbon'}
                }
              )
view.setStyle({'chain':'X'},{'stick':{'colorscheme':'blueCarbon'}})
#Using this line of code you can specify "The text","Opacity of the label","A selection of residues" 
view.addLabel("ETQ",{'fontOpacity':1},{'chain':'X'})
view.show()


**This is the end of the tutorial. Good Science!**